## Intel sensors level A2

Authors: Martin Kostelník, Marianne Jakonen, Ahmed

In [68]:
# Imports
import helper
import pandas as pd
import numpy as np
import math

from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error, r2_score


# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Define some paths
DATA_PATH = r"../data/data.txt.gz"
DATA_PROCESSED_PATH = r"../data/data_processed.csv"

In [3]:
# First let's get the data
# I preprocessed the data and saved it for faster loading

# Load raw data
# data_raw = helper.load_data(DATA_PATH)

# Preprocess data
# data = helper.preprocess_data(data_raw)
# data.to_csv("data_resampled.csv")

In [39]:
# Load processed data
df = pd.read_csv(DATA_PROCESSED_PATH)
df["datetime"] = pd.to_datetime(df["datetime"])

# Find the most correlated variable
correlation = df.corr(numeric_only=True).abs().sum().sort_values(ascending=False)
correlation

H-26    92.736138
H-30    92.091317
H-31    91.160904
H-27    90.419350
H-29    89.718965
          ...    
T-38    44.422759
L-46    43.764384
H-53    39.741264
T-35    36.455761
T-6     32.416700
Length: 156, dtype: float64

We can see that the most correlated variable is humidity of a sensor number 26.
Let's take it out of a dataframe as it is the variable we will make predictions for.

1.3.2004 is Monday, let's start from that

In [73]:
# https://stackoverflow.com/questions/17315737/split-a-large-pandas-dataframe
def split_dataframe(df, chunk_size):
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

r2s = np.zeros((7, 10))

for n_comps in range(1, 11):
    for day in range(1, 8):
        X = df[df["datetime"].dt.date == pd.to_datetime(f"2004-03-0{day}")].loc[:, df.columns != "H-26"].drop(columns=["datetime"])
        Y = df[df["datetime"].dt.date == pd.to_datetime(f"2004-03-0{day}")]["H-26"]

        X_train = X.iloc[:-1, :]
        X_test = X.iloc[-1]
        Y_train = Y.iloc[:-1]
        Y_test = Y.iloc[-1]

        X_train_mean = X_train.mean()
        X_train_std = X_train.std()
        Y_train_mean = Y_train.mean()
        Y_train_std = Y_train.std()

        X_train = (X_train - X_train_mean) / X_train_std
        X_test = (X_test - X_train_mean) / X_train_std
        Y_train = (Y_train - Y_train_mean) / Y_train_std
        Y_test = (Y_test - Y_train_mean) / Y_train_std

        pls = PLSRegression(n_components = n_comps)
        pls.fit(X_train, Y_train)
        Y_pred = pls.predict(np.array(X_test).reshape(1, -1))

        r2 = r2_score(Y_test.squeeze(), Y_pred.squeeze())
        r2s[day, n_comps] = r2

r2s

TypeError: Singleton array 0.2576987471000135 cannot be considered a valid collection.